In [6]:
! pip install tb-nightly

     |████████████████████████████████| 5.8 MB 6.2 MB/s eta 0:00:01
  Using cached tensorboard_plugin_wit-1.8.0-py3-none-any.whl (781 kB)
  Using cached Werkzeug-2.0.2-py3-none-any.whl (288 kB)
  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached grpcio-1.42.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.0 MB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached google_auth-2.3.3-py2.py3-none-any.whl (155 kB)
  Using cached protobuf-3.19.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached cachetools-4.2.4-py3-none-any.whl (10 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2

In [7]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [8]:
class CNN(nn.Module):
    def __init__(self, in_channels = 3, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x
    

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device=", device)

device= cuda


In [26]:
# Hyperparameters

#learning_rate = 0.001
in_channels = 1
num_classes  =10
#batch_size = 64
num_epochs = 1


In [20]:
train_dataset = datasets.MNIST(root='dataset/', train=True, 
                              transform=transforms.ToTensor(), download=True)


In [27]:
batch_sizes = [2, 64, 1024]
learning_rates = [0.1, 0.01, 0.001, 0.0001]


for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        model = CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device)
        model.train()
        criterion = nn.CrossEntropyLoss()
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
        writer = SummaryWriter(f'runs/MNIST/MiniBatchSize {batch_size} LR {learning_rate}')

        step = 0
        for epoch in range(num_epochs):
            losses = []
            accuracies = []

            for batch_idx, (data, targets) in enumerate(train_loader):
                data = data.to(device=device)
                targets = targets.to(device=device)

                #forward
                scores = model(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())

                #backward
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Calculate 'running' training accuracy
                _, predictions = scores.max(1)
                num_correct = (predictions == targets).sum()
                running_train_acc = float(num_correct) / float(data.shape[0])
                accuracies.append(running_train_acc)

                writer.add_scalar('Training loss', loss, global_step = step)
                writer.add_scalar('Training accuracy', running_train_acc, global_step=step)
                step += 1
            
            writer.add_hparams({'lr': learning_rate, 'bsize': batch_size},
                              {'accuracy': sum(accuracies)/len(accuracies),
                              'loss': sum(losses)/len(losses)
                               })
            print(f'Mean loss in this epoch was {sum(losses)/len(losses)}')
        

Mean loss in this epoch was 2.3554092949787777
Mean loss in this epoch was 2.3067014840682347
Mean loss in this epoch was 0.13581883423883342
Mean loss in this epoch was 0.329952061366259
Mean loss in this epoch was 2.3437147082042085
Mean loss in this epoch was 0.15658825641321236
Mean loss in this epoch was 0.35211244492785637
Mean loss in this epoch was 1.1889367956342474
Mean loss in this epoch was 0.6846808078935591
Mean loss in this epoch was 0.47247593104839325
Mean loss in this epoch was 1.4439357802019281
Mean loss in this epoch was 2.271521325838768
